# Connection to MYSQL through Python

So the libraries work correctly it is important to get Python 3.7 or below, also the "sqlalchemy" library is not found in Anaconda or Colab therefore is recommended to run this code through cmd.

Some other important tools:

* Xampp
* MySQL
* Qgis

_The used data for this project can be found in the following link:_ [https://drive.google.com/drive/folders/1Tn8qlEOmWxUU51idEdgRfWRM5k35JtA_?usp=sharing]

In [66]:
#Libraries for the MySQL connection
import csv
import pymysql
import pandas as pd
import unicodedata
from sqlalchemy import create_engine, types
import mysql.connector 

#libraries to use the Mapper algorithm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import sklearn
import datetime as dt 

#Valente
import io
import sys
import base64
import cv2 as cv
import os
import easygui as gui

from io import StringIO 
from sklearn.preprocessing import LabelEncoder 
#Encode target labels with value between 0 and n_classes-1.
from sklearn.preprocessing import StandardScaler 
#Standardize features by removing the mean and scaling to unit variance.

from sklearn.decomposition import PCA 
#Principal component analysis (PCA).
#Linear dimensionality reduction using Singular Value Decomposition of the data to project it to a lower dimensional space. 
#The input data is centered but not scaled for each feature before applying the SVD.

from sklearn.manifold import TSNE
#t-distributed Stochastic Neighbor Embedding. It's a tool to visualize high-dimensional data.

from sklearn import datasets
import umap.umap_ as umap
#Uniform Manifold Approximation and Projection (UMAP) is a dimension reduction technique that can be used for visualisation
#similarly to t-SNE, but also for general non-linear dimension reduction. 

import kmapper as km
from kmapper import jupyter 
#Creates custom CSS full-size Jupyter screen.

from sklearn import preprocessing
#Provides several common utility functions and transformer classes to change raw feature vectors into a representation
#that is more suitable for the downstream estimators.

#Project 
from sklearn import ensemble
from sklearn import manifold

import warnings
warnings.filterwarnings('ignore')

In [67]:
#Testing connection
mydb = mysql.connector.connect( 
  host = "localhost", 
  user = "root", 
  password = "", 
  database = "prueba"
)  
  
mycursor = mydb.cursor() 

INSERTING DATA TO THE DATABASE

* *ITER*

In [63]:
engine = create_engine('mysql://root:@localhost/prueba')
df = pd.read_csv("Entidades2.csv",sep=',',quotechar='\'',encoding='utf-8',error_bad_lines=False) 
df.to_sql('Hidalgo_Municipios',con=engine,index=False,if_exists='append')

* *DRENAJE*

In [9]:
engine = create_engine('mysql://root:@localhost/prueba')
df = pd.read_csv("hidalgo_drenaje.csv",sep=',',quotechar='\'',encoding='utf8') 
df.to_sql('drenaje',con=engine,index=False,if_exists='append')

* *UBICACIÓN*

In [11]:
engine = create_engine('mysql://root:@localhost/prueba')
df = pd.read_csv("ubicacion.csv",sep=',',quotechar='\'',encoding='utf8') 
df.to_sql('ubicación',con=engine,index=False,if_exists='append')

* *COVID*

In [41]:
engine = create_engine('mysql://root:@localhost/prueba')
df = pd.read_csv("Entidadres.csv",sep=',',quotechar='\'',encoding='utf8')
df.to_sql('covid07122020',con=engine,index=False,if_exists='append')

* *DRENAJE SEGUNDA*

In [7]:
engine = create_engine('mysql://root:@localhost/prueba')
df = pd.read_csv("DrenajeN.csv",sep=',',quotechar='\'',encoding='utf8') 
df.to_sql('DrenajeN',con=engine,index=False,if_exists='append')

* SELECTING DATA

In [70]:
#Code to check whether the query is executed successfully or not
''' mycursor.execute("SELECT latitud,longitud FROM covid")
myresult = mycursor.fetchall()
for x in myresult:
  print(x)  '''

# Read the SQL query results using pandas
queryframe = pd.read_sql("""
            SELECT Lon_Decimal, Lat_Decimal, Altitud FROM Hidalgo_municipios LIMIT 10;
            """, con = mydb)

# Return the first five rows
queryframe.head()



,Lon_Decimal,Lat_Decimal,Altitud
0,-98.438372,20.145903,2130
1,-98.203244,20.158693,2255
2,-98.942901,20.269086,2003
3,-98.356386,20.349504,2183
4,-99.122133,20.092525,2143


In [19]:
queryframe.columns

Index(['ID_REGISTRO', 'Nom_Ent', 'Nom_Mun', 'Nom_Loc', 'Lat_Decimal',
       'Lon_Decimal', 'Altitud', 'FECHA_ACTUALIZACION', 'FECHA_INGRESO',
       'FECHA_SINTOMAS', 'INTUBADO', 'UCI'],
      dtype='object')

In [40]:
queryframe.describe()

,Lat_Decimal,Lon_Decimal,Altitud,INTUBADO,UCI
count,10.000000,10.000000,10.000000,10.0,10.0
mean,20.249123,-98.563430,2106.800000,2.0,2.0
std,0.041186,0.047485,73.103883,0.0,0.0
min,20.191681,-98.609821,2026.000000,2.0,2.0
25%,20.214773,-98.601994,2047.500000,2.0,2.0
50%,20.251154,-98.582053,2092.500000,2.0,2.0
75%,20.272658,-98.536302,2165.750000,2.0,2.0
max,20.323425,-98.484039,2227.000000,2.0,2.0


In [18]:
queryframe.shape

(10, 3)

* _CASOS TOTALES_

In [92]:
queryframeTotales = pd.read_sql("""
            SELECT h.Cve_Mun, h.Nom_mun,h.Nom_loc,h.Lat_Decimal,h.Lon_Decimal, COUNT(c.MUNICIPIO_RES) AS Cases
              FROM hidalgo_municipios h INNER JOIN covid07122020 c
              ON h.Cve_Mun = c.MUNICIPIO_RES
              GROUP BY h.Nom_mun;
            """, con = mydb)

# Return the first five rows
queryframeTotales.head()


,Cve_Mun,Nom_mun,Nom_loc,Lat_Decimal,Lon_Decimal,Cases
0,1,Acatlán,Acatlán,20.145903,-98.438372,59
1,2,Acaxochitlán,Acaxochitlán,20.158693,-98.203244,87
2,3,Actopan,Actopan,20.269086,-98.942901,249
3,4,Agua Blanca de Iturbide,Agua Blanca de Iturbide,20.349504,-98.356386,16
4,5,Ajacuba,Ajacuba,20.092525,-99.122133,72


In [ ]:
queryframeTotales.to_csv('C:/Users/almej/Desktop/Totales.csv')

* _MUJERES_

In [87]:
queryframeMujeres = pd.read_sql("""
            SELECT h.Cve_Mun, h.Nom_mun,h.Nom_loc,h.Lat_Decimal,h.Lon_Decimal, COUNT(c.MUNICIPIO_RES) AS Cases
              FROM hidalgo_municipios h INNER JOIN covid07122020 c
              ON h.Cve_Mun = c.MUNICIPIO_RES WHERE c.SEXO ='1'
              GROUP BY h.Nom_mun;
            """, con = mydb)

# Return the first five rows
queryframeMujeres.head()


,Cve_Mun,Nom_mun,Nom_loc,Lat_Decimal,Lon_Decimal,Cases
0,1,Acatlán,Acatlán,20.145903,-98.438372,32
1,2,Acaxochitlán,Acaxochitlán,20.158693,-98.203244,42
2,3,Actopan,Actopan,20.269086,-98.942901,111
3,4,Agua Blanca de Iturbide,Agua Blanca de Iturbide,20.349504,-98.356386,7
4,5,Ajacuba,Ajacuba,20.092525,-99.122133,30


In [ ]:
queryframeMujeres.to_csv('C:/Users/almej/Desktop/Mujeres.csv')

* _HOMBRES_

In [89]:
queryframeHombres = pd.read_sql("""
            SELECT h.Cve_Mun, h.Nom_mun,h.Nom_loc,h.Lat_Decimal,h.Lon_Decimal, COUNT(c.MUNICIPIO_RES) AS Cases
              FROM hidalgo_municipios h INNER JOIN covid07122020 c
              ON h.Cve_Mun = c.MUNICIPIO_RES WHERE c.SEXO ='2'
              GROUP BY h.Nom_mun;
            """, con = mydb)

# Return the first five rows
queryframeHombres.head()


,Cve_Mun,Nom_mun,Nom_loc,Lat_Decimal,Lon_Decimal,Cases
0,1,Acatlán,Acatlán,20.145903,-98.438372,27
1,2,Acaxochitlán,Acaxochitlán,20.158693,-98.203244,45
2,3,Actopan,Actopan,20.269086,-98.942901,138
3,4,Agua Blanca de Iturbide,Agua Blanca de Iturbide,20.349504,-98.356386,9
4,5,Ajacuba,Ajacuba,20.092525,-99.122133,42


In [91]:
queryframeHombres.to_csv('C:/Users/almej/Desktop/Hombres.csv')

* VISUALIZING THE DATAFRAME